In [1]:
import pandas as pd
import xarray as xr
from erddapy import ERDDAP
from datetime import datetime


import sys
sys.path.append('/home/jovyan/repos/hemasphere/botpt/pangeo-forge-recipes')
import pangeo_forge_recipes


from pangeo_forge_recipes.patterns import ConcatDim, FilePattern, MergeDim
from pangeo_forge_recipes.recipes import XarrayZarrRecipe

#pip install fsspec-reference-maker

In [2]:
server = 'http://erddap.dataexplorer.oceanobservatories.org/erddap'
protocol = 'tabledap'

e = ERDDAP(
    server=server,
    protocol=protocol
)

In [3]:
e.dataset_id = 'ooi-rs03ccal-mj03f-05-botpta301'

In [4]:
variables = e.variables = ['time', 'latitude', 'longitude', 'botpres', 'botsflu_meandepth', 'botsflu_meanpres']

In [20]:
years = e.constraints = {
    'time>=': '2014-08-29T00:00:00Z',
    'time<=': '2014-09-02T00:00:00Z',
}

In [21]:
url = e.get_download_url()

In [22]:
url

'http://erddap.dataexplorer.oceanobservatories.org/erddap/tabledap/ooi-rs03ccal-mj03f-05-botpta301.html?time,latitude,longitude,botpres,botsflu_meandepth,botsflu_meanpres&time>=1409270400.0&time<=1409616000.0'

In [23]:
!{'wget '+url}

/bin/bash: =1409616000.0: No such file or directory

real	0m0.001s
user	0m0.000s
sys	0m0.000s


In [24]:
netcdf = e.to_ncCF()

In [25]:
netcdf

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    cdm_data_type: TimeSeries
    cdm_timeseries_variables: station,longitude,latitude
    contributor_email: feedback@axiomdatascience.com
    contributor_name: Axiom Data Science
    contributor_role: processor
    contributor_role_vocabulary: NERC
    contributor_url: https://www.axiomdatascience.com
    Conventions: IOOS-1.2, CF-1.6, ACDD-1.3
    creator_institution: Ocean Observatories Initiative (OOI)
    creator_name: Ocean Observatories Initiative (OOI)
    creator_sector: gov_federal
    creator_type: institution
    creator_url: https://oceanobservatories.org/
    defaultDataQuery: bottilt_tdir,bottilt_ytlt,bottilt_xtlt,bottilt_tmag,bottilt_ccmp,botpres,z,botsflu_predtide,time,bottilt_scmp,botsflu_10minrate,botsflu_5minrate,botsflu_meanpres,botsflu_meandepth&time>=max(time)-3days
    Easternmost_Easting: -130.00876
    featureType: TimeSeries
    geospatial_lat_max: 45.95484
    g

In [29]:
time_concat_dim = ConcatDim("time", years, nitems_per_file=1)
time_concat_dim

ConcatDim(name='time', nitems_per_file=1)

In [30]:
def make_url(dataset_id, variable, time):
    return f"http://erddap.dataexplorer.oceanobservatories.org/erddap/tabledap/{dataset_id}.nc?{variables}&time%3E={datetime.fromtimestamp(time)}&time%3C={datetime.fromtimestamp(time)}"

In [31]:
pattern = FilePattern(make_url, time_concat_dim)
pattern

<FilePattern {'time': 2}>

In [28]:
for index, url in pattern.items():
    print(index)
    print(url)
    if datetime.timestamp(years) in url:
        break
    

KeyError: 0

In [ ]:
test = xr.open_dataset("test.nc")
test2 = xr.open_dataset("test2.nc")
# wget -O 

In [ ]:
test

In [ ]:
test2